# Mini Proyecto

datos: https://www.cs.cmu.edu/~ark/personas/

In [3]:
# Librerias

import pandas as pd
import pyspark
# Cargar Funciones Extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Proyecto_1').getOrCreate()

22/02/25 17:51:00 WARN Utils: Your hostname, MacBook-Air-de-Daniel.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
22/02/25 17:51:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 17:51:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
#Carga de bibliotecas extras
import numpy as np
import matplotlib.pyplot as plt

#Carga de NLTK
import nltk
import string

In [44]:
def read_tsv(direccion):
    import csv
    tsv_file = open(direccion)
    read_tsv = csv.reader(tsv_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    tsv_file.close()
    return(spark.sparkContext.parallelize(lista))

def read_txt(direccion):
    import csv
    txt_file = open(direccion)
    read_tsv = csv.reader(txt_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    txt_file.close()
    return(spark.sparkContext.parallelize(lista))

In [45]:
# Importamos datos:

character = read_tsv('MovieSummaries/character.metadata.tsv')
movie = read_tsv('MovieSummaries/movie.metadata.tsv')

In [51]:
name = read_txt('MovieSummaries/name.clusters.txt')
tvtropes = read_txt('MovieSummaries/tvtropes.clusters.txt')
plot_summaries =read_txt('MovieSummaries/plot_summaries.txt')

¿Cual es género más visto?

In [125]:
def sep_gen(renglon):
    renglon_separado = renglon[8]
    import re
    generos = re.findall(r'"([\w\s]+)"', renglon_separado)
    for g in generos:
        yield g

In [139]:
genere = movie.flatMap(sep_gen)
genere = genere.map(lambda renglon: (renglon, 1))
# Conteo
genere_count = genere.reduceByKey(lambda val_1 , val_2: val_1 + val_2).sortBy(lambda renglon: renglon[1], ascending = False)

22/02/26 00:37:11 WARN TaskSetManager: Stage 61 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


In [140]:
genere_count.take(10)

[('Drama', 34007),
 ('Comedy', 16349),
 ('Romance Film', 10234),
 ('Action', 8798),
 ('Thriller', 8744),
 ('Short Film', 8141),
 ('World cinema', 7155),
 ('Crime Fiction', 6948),
 ('Indie', 6897),
 ('Documentary', 5630)]